# Assignment 04

Caroline O’Sullivan (Boston University)  
September 27, 2025

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import plotly.express as px
import plotly.io as pio
import numpy as np

np.random.seed(45)

pio.renderers.default = "notebook+notebook_connected+vscode"

# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load Data
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("./data/lightcast_job_postings.csv")

# Show Schema and Sample Data
#print("---This is Diagnostic check, No need to print it in the final doc---")

# df.printSchema() # comment this line when rendering the submission
df.show(5)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/01 00:47:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/10/01 00:48:04 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-----------------+----------------------+----------+--------+---------+--------+--------------------+--------------------+--------------------+-----------+-------------------+--------------------+--------------------+---------------+----------------+--------+--------------------+-----------+-------------------+----------------+---------------------+-------------+-------------------+-------------+------------------+---------------+--------------------+--------------------+--------------------+-------------+------+-----------+----------------+-------------------+---------+-----------+--------------------+--------------------+-------------+------+--------------+-----+--------------------+-----+----------+---------------+--------------------+---------------+--------------------+------------+--------------------+------------+--------------------+------+--------------------+------+--------------------+------+--------------------+------+--------------------+------+------

In [5]:

df.select("EMPLOYMENT_TYPE_NAME").distinct().show()

+--------------------+
|EMPLOYMENT_TYPE_NAME|
+--------------------+
|Part-time / full-...|
|Part-time (â‰¤ 32...|
|Full-time (> 32 h...|
|                NULL|
+--------------------+



##Feature Engineering
Feature Engineering is a crucial step in preparing your data for machine learning. In this lab, we will focus on the following tasks:

1. Drop rows with missing values in the target variable and key features.
2. By now you are already familiar with the code and the data. Based on your understanding please choose any 3 (my code output has 10) variables as:
three continuous variables and, MIN_YEARS_EXPERIENCE (total 4, use your best judgment!)
two categorical .
Your dependent variable (y) is SALARY.
3. Convert categorical variables into numerical representations using StringIndexer and OneHotEncoder.
4. Assemble features into a single vector using VectorAssembler.
5. Split the data into training and testing sets.
6. You can use pipeline to do the above steps in one go.
7. Create a new column MIN_YEARS_EXPERIENCE_SQ by squaring the MIN_YEARS_EXPERIENCE column.
8. Assemble the polynomial features into a new vector column features_poly using VectorAssembler.
9. Show the final structure of the DataFrame with the new features.
#Missing Value Treatment
1. Replace missing values in Salary by Median of Salary based on the Employment Type, if missing then replace with the overall median of Salary

Continous:
MIN_YEARS_EXPERIENCE


Categorical:
STATE_NAME
REMOTE_TYPE_NAME

Dependent (Y)
SALARY

In [ ]:
#Missing Value Treatment
#1. Replace missing values in Salary by Median of Salary based on the Employment Type, if missing then replace with the overall median of Salary

from pyspark.sql import Window
from pyspark.sql.functions import col, when, isnan, count, lit, expr, avg, median, pow
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

#select subset of columns
eda_columns = ["EMPLOYMENT_TYPE", "SALARY", "MIN_YEARS_EXPERIENCE", "CITY_NAME", "REMOTE_TYPE_NAME", "STATE_NAME","COMPANY_IS_STAFFING","TITLE_RAW","MINIMUM_EDUCATION_LEVEL"]

#Fix Remote Type Name incorrect labels to Remote, Hybrid, Onsite.  None and NULL are Onsite.
df_subset = df_subset.withColumn(
    "REMOTE_TYPE_NAME",
    when(col("REMOTE_TYPE_NAME") == "Remote", "Remote")
    .when(col("REMOTE_TYPE_NAME") == "Not Remote", "On Site")
    .when(col("REMOTE_TYPE_NAME") == "Hybrid Remote", "Hybrid")
    .when((col("REMOTE_TYPE_NAME").isNull()) | (col("REMOTE_TYPE_NAME") == "[None]"), "On Site")
    .otherwise("On Site")
)

#Clean Employment Type
df_subset = df_subset.withColumn(
    "EMPLOYMENT_TYPE_NAME",
    when(col("EMPLOYMENT_TYPE_NAME").like("Part-time (â‰¤ 32%"), "Part Time")
    .when(col("EMPLOYMENT_TYPE_NAME").like("Part-time / full-%"), "Flexible")
    .when(col("EMPLOYMENT_TYPE_NAME").like("Full-time (> 32%"), "Full Time")
    .when(col("EMPLOYMENT_TYPE_NAME").isNull(), "Full Time")
    .otherwise("Full Time")  # fallback for anything else
)
#Calculate the median of MIN_YEARS_EXPERIENCE column based on Employment Type
median_by_emp_type_min_exp = df.groupBy("EMPLOYMENT_TYPE").agg(expr('percentile_approx(MIN_YEARS_EXPERIENCE, 0.5)').alias('median_min_years_experience_by_emp_type'))

#Median Salary
median_salary = df.approxQuantile("SALARY", [0.5], 0.25)[0]
median_by_emp_type = df.groupBy("EMPLOYMENT_TYPE").agg(expr('percentile_approx(SALARY, 0.5)').alias('median_salary_by_emp_type'))

df_salary_imputed = df.join(median_by_emp_type, on="EMPLOYMENT_TYPE", how="left")
df_salary_imputed = df_salary_imputed.withColumn(
    "SALARY",
    when(
        col("SALARY").isNull(),
        when(col("median_salary_by_emp_type").isNotNull(), col("median_salary_by_emp_type"))
        .otherwise(lit(median_salary))
    ).otherwise(col("SALARY"))
)
df_salary_imputed.show(5)

NameError: name 'df_subset' is not defined

In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import col, when, lit, expr

# 1️⃣ Define subset of columns you care about
eda_columns = [
    "EMPLOYMENT_TYPE",
    "EMPLOYMENT_TYPE_NAME",
    "SALARY",
    "MIN_YEARS_EXPERIENCE",
    "CITY_NAME",
    "REMOTE_TYPE_NAME",
    "STATE_NAME",
    "COMPANY_IS_STAFFING",
    "TITLE_RAW",
    "MINIMUM_EDUCATION_LEVEL"
]

# 2️⃣ Create subset DataFrame
df_subset = df.select(*eda_columns)

# 3️⃣ Clean REMOTE_TYPE_NAME
df_subset = df_subset.withColumn(
    "REMOTE_TYPE_NAME",
    when(col("REMOTE_TYPE_NAME") == "Remote", "Remote")
    .when(col("REMOTE_TYPE_NAME") == "Not Remote", "On Site")
    .when(col("REMOTE_TYPE_NAME") == "Hybrid Remote", "Hybrid")
    .when((col("REMOTE_TYPE_NAME").isNull()) | (col("REMOTE_TYPE_NAME") == "[None]"), "On Site")
    .otherwise("On Site")
)

# 4️⃣ Clean EMPLOYMENT_TYPE_NAME
df_subset = df_subset.withColumn(
    "EMPLOYMENT_TYPE_NAME",
    when(col("EMPLOYMENT_TYPE_NAME").like("Part-time (â‰¤ 32%"), "Part Time")
    .when(col("EMPLOYMENT_TYPE_NAME").like("Part-time / full-%"), "Flexible")
    .when(col("EMPLOYMENT_TYPE_NAME").like("Full-time (> 32%"), "Full Time")
    .when(col("EMPLOYMENT_TYPE_NAME").isNull(), "Full Time")
    .otherwise("Full Time")
)

# 5️⃣ Compute median salary
median_salary = df.approxQuantile("SALARY", [0.5], 0.25)[0]

# 6️⃣ Compute median SALARY by EMPLOYMENT_TYPE
median_by_emp_type_salary = df.groupBy("EMPLOYMENT_TYPE").agg(
    expr('percentile_approx(SALARY, 0.5)').alias('median_salary_by_emp_type')
)

# 7️⃣ Compute median MIN_YEARS_EXPERIENCE by EMPLOYMENT_TYPE
median_by_emp_type_min_exp = df.groupBy("EMPLOYMENT_TYPE").agg(
    expr('percentile_approx(MIN_YEARS_EXPERIENCE, 0.5)').alias('median_min_years_experience_by_emp_type')
)

# 8️⃣ Join median values with subset
df_subset = df_subset.join(median_by_emp_type_salary, on="EMPLOYMENT_TYPE", how="left")
df_subset = df_subset.join(median_by_emp_type_min_exp, on="EMPLOYMENT_TYPE", how="left")

# 9️⃣ Impute missing SALARY
df_subset = df_subset.withColumn(
    "SALARY",
    when(
        col("SALARY").isNull(),
        when(col("median_salary_by_emp_type").isNotNull(), col("median_salary_by_emp_type"))
        .otherwise(lit(median_salary))
    ).otherwise(col("SALARY"))
)

#  🔟 Impute missing MIN_YEARS_EXPERIENCE
df_subset = df_subset.withColumn(
    "MIN_YEARS_EXPERIENCE",
    when(
        col("MIN_YEARS_EXPERIENCE").isNull(),
        when(col("median_min_years_experience_by_emp_type").isNotNull(), col("median_min_years_experience_by_emp_type"))
        .otherwise(lit(0))  # fallback if needed
    ).otherwise(col("MIN_YEARS_EXPERIENCE"))
)

# 11️⃣ Drop helper median columns
df_subset = df_subset.drop("median_salary_by_emp_type", "median_min_years_experience_by_emp_type")

# 12️⃣ Preview cleaned subset
df_subset.show(5)
